In [ ]:
import json
import os
import glob
import tqdm
import pprint
import collections
import random
import numpy as np

from together import Together

In [ ]:
with open('./prompts/r1_zero.prompt') as f:
    prompt_template = f.read()
print(prompt_template)

A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: {question}
Assistant: <think>


In [3]:
train_examples = []
for fname in tqdm.tqdm(glob.glob("../data/MATH/train/*/*json")):
    with open(fname, "r") as f:
        train_examples.append(json.load(f))
print(len(train_examples))
with open("../data/MATH/train.jsonl", "w") as f:
    for example in train_examples:
        f.write(json.dumps(example) + "\n")
with open("../data/MATH/sft.jsonl", "w") as f:
    for example in train_examples:
        sft_example = {
            "prompt": prompt_template.format(question=example["problem"]),
            "response": example["solution"],
        }
        f.write(json.dumps(sft_example) + "\n")

test_examples = []
for fname in tqdm.tqdm(glob.glob("../data/MATH/test/*/*json")):
    with open(fname, "r") as f:
        test_examples.append(json.load(f))
print(len(test_examples))
with open("../data/MATH/validation.jsonl", "w") as f:
    for example in test_examples:
        f.write(json.dumps(example) + "\n")

100%|██████████| 7500/7500 [00:01<00:00, 6869.08it/s]


7500


100%|██████████| 5000/5000 [00:00<00:00, 6613.61it/s]

5000


In [ ]:
# zero-shot
import json
import collections

results = []
with open('../eval_result.jsonl') as f:
    for line in f.readlines():
        results.append(json.loads(line))

counter = collections.Counter()
for res in results:
    if res['format_reward'] > 0 and res['answer_reward'] > 0:
        counter['f1_a1'] += 1
    elif res['format_reward'] > 0:
        counter['f1_a0'] += 1
    elif res['answer_reward'] > 0:
        counter['f0_a1'] += 1
    else:
        counter['f0_a0'] += 1

print(counter)
print('format accuracy', (counter['f1_a0'] + counter['f1_a1']) / sum(counter.values()))
print('answer accuracy', (counter['f1_a1']) / sum(counter.values()))


Counter({'f0_a0': 4186, 'f1_a0': 691, 'f1_a1': 123})
format accuracy 0.1628
answer accuracy 0.0246


In [ ]:
# expert iteration
import json
import collections

results = []
with open('../experiments/02-07-2025-00-15-20/eval_results-1.jsonl') as f:
    for line in f.readlines():
        results.append(json.loads(line))

counter = collections.Counter()
for res in results:
    if res['format_reward'] > 0 and res['answer_reward'] > 0:
        counter['f1_a1'] += 1
    elif res['format_reward'] > 0:
        counter['f1_a0'] += 1
    elif res['answer_reward'] > 0:
        counter['f0_a1'] += 1
    else:
        counter['f0_a0'] += 1

print(counter)
print('format accuracy', (counter['f1_a0'] + counter['f1_a1']) / sum(counter.values()))
print('answer accuracy', (counter['f1_a1']) / sum(counter.values()))

Counter({'f1_a0': 2675, 'f1_a1': 2124, 'f0_a0': 201})
format accuracy 0.9598
answer accuracy 0.4248
